<a href="https://colab.research.google.com/github/hikaruyaku/KIKAGAKU-/blob/master/5_6_Optuna_%E3%81%AB%E3%82%88%E3%82%8B%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%83%BC%E3%81%AE%E6%9C%80%E9%81%A9%E5%8C%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Optuna によるハイパーパラメーターの最適化

### 1. ハイパーパラメーターの最適化

- グリッドサーチ，ランダムサーチ → 事前に最適化を行うハイパーパラメータの組み合わせを決定
- モデルベースの探索 → すでに行われた試行に対する結果基づいて次の試行を行う

ベイズ最適化(Bayesian optimization)

In [1]:
!pip install -q optuna

     |████████████████████████████████| 266kB 9.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 15.5MB/s 
     |████████████████████████████████| 81kB 7.0MB/s 
     |████████████████████████████████| 81kB 7.3MB/s 
     |████████████████████████████████| 133kB 15.8MB/s 
     |████████████████████████████████| 112kB 16.8MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 


In [2]:
import optuna
optuna.__version__

'2.3.0'

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.__version__

'1.7.0+cu101'

In [4]:
torch.cuda.get_device_name(0)

'Tesla P100-PCIE-16GB'

In [8]:
!pip install -q pytorch_lightning==1.0.8
import pytorch_lightning as pl
pl.__version__

     |████████████████████████████████| 563kB 8.4MB/s 
     |████████████████████████████████| 276kB 16.1MB/s 
     |████████████████████████████████| 829kB 15.8MB/s 
     |████████████████████████████████| 102kB 9.4MB/s 


'1.0.8'

### 2. Optuna による最適化入門

$$
f(x) = (x-2)^{2}
$$

の最小化問題を解きます。最適解は $x=2$

In [9]:
def objective(trial):

  x = trial.suggest_uniform('x',-10,10)

  return (x-2)**2

In [10]:
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler)
study.optimize(objective,n_trials = 10)

[I 2021-01-06 07:05:18,480] A new study created in memory with name: no-name-e0a4e230-bf1b-4801-b23c-956ae43b1555
[I 2021-01-06 07:05:18,488] Trial 0 finished with value: 1.0480229520791995 and parameters: {'x': 0.9762700785464951}. Best is trial 0 with value: 1.0480229520791995.
[I 2021-01-06 07:05:18,493] Trial 1 finished with value: 5.307436050111796 and parameters: {'x': 4.30378732744839}. Best is trial 0 with value: 1.0480229520791995.
[I 2021-01-06 07:05:18,497] Trial 2 finished with value: 0.0030544989253336228 and parameters: {'x': 2.055267521432878}. Best is trial 2 with value: 0.0030544989253336228.
[I 2021-01-06 07:05:18,500] Trial 3 finished with value: 1.2151454066214245 and parameters: {'x': 0.8976636599379368}. Best is trial 2 with value: 0.0030544989253336228.
[I 2021-01-06 07:05:18,504] Trial 4 finished with value: 12.439051918480782 and parameters: {'x': -1.5269040132219054}. Best is trial 2 with value: 0.0030544989253336228.
[I 2021-01-06 07:05:18,507] Trial 5 finish

In [13]:
study.best_params

{'x': 2.055267521432878}

### 3. データセットの準備

In [14]:
# データセットの読込
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
x = breast_cancer['data']
t = breast_cancer['target']

In [15]:
# Pytorch の Tensor 型への変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [16]:
type(x),type(t)

(torch.Tensor, torch.Tensor)

In [17]:
x.dtype,t.dtype

(torch.float32, torch.int64)

In [18]:
# dataset にまとめる
dataset = torch.utils.data.TensorDataset(x,t)
dataset

In [19]:
# ランダムにデータセットを分割
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

n_train,n_val,n_test
# 乱数のシードを固定
pl.seed_everything(0)

# データセットの分割
train,val,test = torch.utils.data.random_split(dataset,[n_train,n_val,n_test])

In [20]:
len(train),len(val),len(test)

(341, 113, 115)

In [21]:
# DataLoader
batch_size = 32

train_loader = torch.utils.data.DataLoader(train,batch_size, shuffle=True,drop_last=True)
val_loader = torch.utils.data.DataLoader(val,batch_size)
test_loader = torch.utils.data.DataLoader(test,batch_size)

#### 4. 学習係数を最適化

In [22]:
from pytorch_lightning.metrics.functional import  accuracy

In [23]:
class Net(pl.LightningModule):
  def __init__(self, lr=0.01):
    super().__init__()

    self.lr = lr

    self.fc1 = nn.Linear(30,10)
    self.bn1 = nn.BatchNorm1d(10,10)
    self.fc2 = nn.Linear(10,2)

  def forward(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.bn1(h)
    h = self.fc2(h)
    return h

  def training_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('train_loss',loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log('train_acc',accuracy(y,t), on_step=False, on_epoch=True, prog_bar=True)
    return loss

  # 検証データに対する処理
  def validation_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('val_loss',loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log('val_acc',accuracy(y,t), on_step=False, on_epoch=True, prog_bar=True)

  # テストデータに対する処理
  def test_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('test_loss',loss, on_step=False, on_epoch=True)
    self.log('test_acc',accuracy(y,t), on_step=False, on_epoch=True)



  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(),lr=self.lr)
    return optimizer

In [26]:
from pytorch_lightning.callbacks import  EarlyStopping

In [29]:
def objective(trial):

  # 学習係数
  lr = trial.suggest_loguniform('lr',1e-5,1e-1)

  # 目的関数；検証データに対する正解率 (accuary)
  pl.seed_everything(0)
  net = Net(lr=lr)
  trainer = pl.Trainer(max_epochs=10,gpus=1,deterministic=True,callbacks=[EarlyStopping(monitor='val_acc')])
  trainer.fit(net, train_loader,val_loader)

  return trainer.callback_metrics['val_acc']

In [ ]:
# ハイパーパラメータの最適化

sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective,n_trials = 30)

In [36]:
study.best_trial

FrozenTrial(number=7, value=0.9203540086746216, datetime_start=datetime.datetime(2021, 1, 6, 7, 36, 29, 941694), datetime_complete=datetime.datetime(2021, 1, 6, 7, 36, 31, 82637), params={'lr': 0.03690557729213761}, distributions={'lr': LogUniformDistribution(high=0.1, low=1e-05)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=7, state=TrialState.COMPLETE)

In [37]:
study.best_params

{'lr': 0.03690557729213761}

In [38]:
study.best_value

0.9203540086746216

In [39]:
# 試行結果の可視化
optuna.visualization.plot_optimization_history(study)

In [40]:
# パラメータと目的関数の値の関係性を可視化
optuna.visualization.plot_slice(study)

### 5. 中間層のノード数を最適化

In [43]:
class Net(pl.LightningModule):
  def __init__(self, n_mid = 10,lr=0.01):
    super().__init__()

    self.lr = lr
    self.n_mid = n_mid

    self.fc1 = nn.Linear(30,self.n_mid)
    self.bn1 = nn.BatchNorm1d(self.n_mid)
    self.fc2 = nn.Linear(self.n_mid,2)

  def forward(self, x):
    h = self.fc1(x)
    h = F.relu(h)
    h = self.bn1(h)
    h = self.fc2(h)
    return h

  def training_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('train_loss',loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log('train_acc',accuracy(y,t), on_step=False, on_epoch=True, prog_bar=True)
    return loss

  # 検証データに対する処理
  def validation_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('val_loss',loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log('val_acc',accuracy(y,t), on_step=False, on_epoch=True, prog_bar=True)

  # テストデータに対する処理
  def test_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('test_loss',loss, on_step=False, on_epoch=True)
    self.log('test_acc',accuracy(y,t), on_step=False, on_epoch=True)



  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(),lr=self.lr)
    return optimizer

In [48]:
def objective(trial):

  # 学習係数
  lr = trial.suggest_loguniform('lr',1e-5,1e-1)

  # 中間層のノード数
  n_mid = trial.suggest_int('n_mid',2,100)

  # 目的関数；検証データに対する正解率 (accuary)
  pl.seed_everything(0)
  net = Net(n_mid=n_mid, lr=lr)
  trainer = pl.Trainer(max_epochs=10,gpus=1,deterministic=True,callbacks=[EarlyStopping(monitor='val_acc')])
  trainer.fit(net, train_loader,val_loader)

  return trainer.callback_metrics['val_acc']

In [49]:
# ハイパーパラメータの最適化

sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective,n_trials = 30)

[I 2021-01-06 07:44:59,406] A new study created in memory with name: no-name-72def494-83ab-4d03-aaca-85d850e34530
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning:

You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 2.0 K 
1 | bn1  | BatchNorm1d | 132   
2 | fc2  | Linear      | 134   


[I 2021-01-06 07:45:00,695] Trial 0 finished with value: 0.9115044474601746 and parameters: {'lr': 0.0015676677195506068, 'n_mid': 66}. Best is trial 0 with value: 0.9115044474601746.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 2.1 K 
1 | bn1  | BatchNorm1d | 138   
2 | fc2  | Linear      | 140   


[I 2021-01-06 07:45:01,872] Trial 1 finished with value: 0.8761062026023865 and parameters: {'lr': 0.0025766385746135885, 'n_mid': 69}. Best is trial 0 with value: 0.9115044474601746.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 2.6 K 
1 | bn1  | BatchNorm1d | 170   
2 | fc2  | Linear      | 172   


[I 2021-01-06 07:45:03,216] Trial 2 finished with value: 0.9203540086746216 and parameters: {'lr': 0.024491024634012086, 'n_mid': 85}. Best is trial 2 with value: 0.9203540086746216.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 1.2 K 
1 | bn1  | BatchNorm1d | 76    
2 | fc2  | Linear      | 78    


[I 2021-01-06 07:45:04,480] Trial 3 finished with value: 0.8053097128868103 and parameters: {'lr': 0.0038333321561566623, 'n_mid': 38}. Best is trial 2 with value: 0.9203540086746216.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 2.8 K 
1 | bn1  | BatchNorm1d | 180   
2 | fc2  | Linear      | 182   


...

[I 2021-01-06 07:45:30,908] Trial 25 finished with value: 0.9115044474601746 and parameters: {'lr': 0.09109639922217964, 'n_mid': 34}. Best is trial 14 with value: 0.9380530714988708.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 2.7 K 
1 | bn1  | BatchNorm1d | 174   
2 | fc2  | Linear      | 176   


[I 2021-01-06 07:45:32,157] Trial 26 finished with value: 0.9026548862457275 and parameters: {'lr': 0.03433437337239421, 'n_mid': 87}. Best is trial 14 with value: 0.9380530714988708.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 1.8 K 
1 | bn1  | BatchNorm1d | 118   
2 | fc2  | Linear      | 120   


[I 2021-01-06 07:45:33,273] Trial 27 finished with value: 0.8938053250312805 and parameters: {'lr': 0.09617284006216398, 'n_mid': 59}. Best is trial 14 with value: 0.9380530714988708.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 1.5 K 
1 | bn1  | BatchNorm1d | 96    
2 | fc2  | Linear      | 98    


[I 2021-01-06 07:45:34,435] Trial 28 finished with value: 0.9203540086746216 and parameters: {'lr': 0.003762539390614542, 'n_mid': 48}. Best is trial 14 with value: 0.9380530714988708.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type        | Params
-------------------------------------
0 | fc1  | Linear      | 2.1 K 
1 | bn1  | BatchNorm1d | 134   
2 | fc2  | Linear      | 136   


[I 2021-01-06 07:45:35,605] Trial 29 finished with value: 0.9026548862457275 and parameters: {'lr': 0.02094652234041386, 'n_mid': 67}. Best is trial 14 with value: 0.9380530714988708.


In [50]:
study.best_params

{'lr': 0.02680770488116905, 'n_mid': 86}

In [51]:
study.best_value

0.9380530714988708

In [53]:
optuna.visualization.plot_optimization_history(study)

In [54]:
# 2つのパラメータの関係性を等高線でプロットする
optuna.visualization.plot_contour(study)

### 6. 中間層の数を最適化

In [55]:
layers = nn.Sequential()
layers

Sequential()

In [56]:
fc = nn.Linear(10,10)

In [57]:
layers.add_module('fc1',fc)

In [58]:
layers

Sequential(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
)

In [64]:
layers = nn.Sequential()
for n in range(3):
  layers.add_module(f'fc{n+1}',nn.Linear(30,30))
  layers.add_module(f'bn{n+1}',nn.BatchNorm1d(30))

In [65]:
layers

Sequential(
  (fc1): Linear(in_features=30, out_features=30, bias=True)
  (bn1): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=30, out_features=30, bias=True)
  (bn2): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=30, out_features=30, bias=True)
  (bn3): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [61]:
x.shape

torch.Size([569, 30])

In [63]:
h = layers(x)
h.shape

torch.Size([569, 30])

In [66]:
layers = nn.Sequential()
for n in range(3):
  layers.add_module(f'fc{n+1}',nn.Linear(30,30))
  layers.add_module(f'act{n+1}',nn.ReLU())
  layers.add_module(f'bn{n+1}',nn.BatchNorm1d(30))

In [67]:
layers

Sequential(
  (fc1): Linear(in_features=30, out_features=30, bias=True)
  (act1): ReLU()
  (bn1): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=30, out_features=30, bias=True)
  (act2): ReLU()
  (bn2): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=30, out_features=30, bias=True)
  (act3): ReLU()
  (bn3): BatchNorm1d(30, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [68]:
y = layers(x)
y.shape

torch.Size([569, 30])

In [80]:
class Net(pl.LightningModule):
  def __init__(self,n_layers=1, n_mid = 10,lr=0.01):
    super().__init__()

    self.lr = lr
    self.n_mid = n_mid
    self.n_layers = n_layers

    self.layers = nn.Sequential()
    for n in range(self.n_layers):
      if n == 0:
        self.layers.add_module(f'fc{n+1}', nn.Linear(30,self.n_mid))
      else:
        self.layers.add_module(f'fc{n+1}', nn.Linear(self.n_mid,self.n_mid))
      self.layers.add_module(f'act{n+1}', nn.ReLU())
      self.layers.add_module(f'bn{n+1}',nn.BatchNorm1d(self.n_mid))

  # 最後の層
    self.layers.add_module(f'fc{self.n_layers+1}',nn.Linear(self.n_mid, 2))

  def forward(self, x):
    h = self.layers(x)
    return h

  def training_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('train_loss',loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log('train_acc',accuracy(y,t), on_step=False, on_epoch=True, prog_bar=True)
    return loss

  # 検証データに対する処理
  def validation_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('val_loss',loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log('val_acc',accuracy(y,t), on_step=False, on_epoch=True, prog_bar=True)

  # テストデータに対する処理
  def test_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('test_loss',loss, on_step=False, on_epoch=True)
    self.log('test_acc',accuracy(y,t), on_step=False, on_epoch=True)



  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(),lr=self.lr)
    return optimizer

Net(
  (layers): Sequential(
    (fc1): Linear(in_features=30, out_features=10, bias=True)
    (act1): ReLU()
    (bn1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc2): Linear(in_features=10, out_features=10, bias=True)
    (act2): ReLU()
    (bn2): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc3): Linear(in_features=10, out_features=10, bias=True)
    (act3): ReLU()
    (bn3): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (fc4): Linear(in_features=10, out_features=2, bias=True)
  )
)

In [81]:
def objective(trial):

  # ハイパーパラメータの候補領域
  n_layers = trial.suggest_int('n_layers',1,5)
  lr = trial.suggest_loguniform('lr',1e-5,1e-1)
  n_mid = trial.suggest_int('n_mid',2,100)

  # 目的関数；検証データに対する正解率 (accuary)
  pl.seed_everything(0)
  net = Net(n_layers = n_layers,n_mid=n_mid, lr=lr)
  trainer = pl.Trainer(max_epochs=10,gpus=1,deterministic=True,callbacks=[EarlyStopping(monitor='val_acc')])
  trainer.fit(net, train_loader,val_loader)

  return trainer.callback_metrics['val_acc']

In [82]:
# ハイパーパラメータの最適化

sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(sampler=sampler, direction='maximize')
study.optimize(objective,n_trials = 30)

[I 2021-01-06 08:06:42,364] A new study created in memory with name: no-name-71cfc4cc-b240-4c9d-91ee-e4c468ace072
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning:

You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 20.5 K


[I 2021-01-06 08:06:43,893] Trial 0 finished with value: 0.9380530714988708 and parameters: {'n_layers': 5, 'lr': 0.002351681338577184, 'n_mid': 66}. Best is trial 0 with value: 0.9380530714988708.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 849   


[I 2021-01-06 08:06:45,276] Trial 1 finished with value: 0.9203540086746216 and parameters: {'n_layers': 4, 'lr': 0.027026043440193118, 'n_mid': 11}. Best is trial 0 with value: 0.9380530714988708.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 6.0 K 


[I 2021-01-06 08:06:47,016] Trial 2 finished with value: 0.9115044474601746 and parameters: {'n_layers': 4, 'lr': 0.0038333321561566623, 'n_mid': 38}. Best is trial 0 with value: 0.9380530714988708.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 2.1 K 


[I 2021-01-06 08:06:48,135] Trial 3 finished with value: 0.9203540086746216 and parameters: {'n_layers': 1, 'lr': 0.07155682161754869, 'n_mid': 60}. Best is trial 0 with value: 0.9380530714988708.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 11.3 K


...

[I 2021-01-06 08:07:19,212] Trial 25 finished with value: 0.7964601516723633 and parameters: {'n_layers': 4, 'lr': 0.03443543874098255, 'n_mid': 44}. Best is trial 12 with value: 0.9469026327133179.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 15.7 K


[I 2021-01-06 08:07:20,513] Trial 26 finished with value: 0.9203540086746216 and parameters: {'n_layers': 3, 'lr': 0.006736556742428521, 'n_mid': 79}. Best is trial 12 with value: 0.9469026327133179.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 18.8 K


[I 2021-01-06 08:07:21,851] Trial 27 finished with value: 0.9203540086746216 and parameters: {'n_layers': 5, 'lr': 0.021002400048439136, 'n_mid': 63}. Best is trial 12 with value: 0.9469026327133179.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 10.8 K


[I 2021-01-06 08:07:23,255] Trial 28 finished with value: 0.9292035102844238 and parameters: {'n_layers': 4, 'lr': 0.04172231399314808, 'n_mid': 53}. Best is trial 12 with value: 0.9469026327133179.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type       | Params
--------------------------------------
0 | layers | Sequential | 2.1 K 


[I 2021-01-06 08:07:25,037] Trial 29 finished with value: 0.6548672318458557 and parameters: {'n_layers': 4, 'lr': 0.0016473634651303275, 'n_mid': 20}. Best is trial 12 with value: 0.9469026327133179.


In [83]:
study.best_params

{'lr': 0.00718497950844088, 'n_layers': 5, 'n_mid': 68}

In [84]:
study.best_value

0.9469026327133179

In [85]:
optuna.visualization.plot_optimization_history(study)

In [86]:
optuna.visualization.plot_contour(study)

In [ ]:
class Net(pl.LightningModule):
  def __init__(self,n_layers=1, n_mid = 10,lr=0.01):
    super().__init__()

    self.lr = lr
    self.n_mid = n_mid
    self.n_layers = n_layers

    self.layers = nn.Sequential()
    for n in range(self.n_layers):
      if n == 0:
        self.layers.add_module(f'fc{n+1}', nn.Linear(30,self.n_mid))
      else:
        self.layers.add_module(f'fc{n+1}', nn.Linear(self.n_mid,self.n_mid))
      self.layers.add_module(f'act{n+1}', nn.ReLU())
      self.layers.add_module(f'bn{n+1}',nn.BatchNorm1d(self.n_mid))

  # 最後の層
    self.layers.add_module(f'fc{self.n_layers+1}',nn.Linear(self.n_mid, 2))

  def forward(self, x):
    h = self.layers(x)
    return h

  def training_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('train_loss',loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log('train_acc',accuracy(y,t), on_step=False, on_epoch=True, prog_bar=True)
    return loss

  # 検証データに対する処理
  def validation_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('val_loss',loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log('val_acc',accuracy(y,t), on_step=False, on_epoch=True, prog_bar=True)

  # テストデータに対する処理
  def test_step(self,batch,batch_idx):
    x,t = batch
    y = self(x)
    loss = F.cross_entropy(y,t)
    self.log('test_loss',loss, on_step=False, on_epoch=True)
    self.log('test_acc',accuracy(y,t), on_step=False, on_epoch=True)



  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(),lr=self.lr)
    return optimizer